In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('datasets/WEB_LOG_DATA.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5866 entries, 0 to 5865
Data columns (total 6 columns):
ip           5866 non-null object
date_time    5866 non-null object
request      5866 non-null object
step         5866 non-null int64
session      5866 non-null int64
user_id      5866 non-null int64
dtypes: int64(3), object(3)
memory usage: 275.0+ KB


In [2]:
df.head()

ip             date_time request  step  session  user_id
0   c210-49-32-6.rochd2.  18/Aug/2017:21:25:07       /     1        3        3
1   visp.inabox.telstra.  19/Aug/2017:08:24:28       /     1       12       12
2  adsl-61-95-54-84.requ  19/Aug/2017:08:33:01       /     1       13       13
3  d220-236-91-52.adsl.n  19/Aug/2017:09:16:06       /     1       15       15
4      allptrs.eq.edu.au  19/Aug/2017:09:47:54       /     1       22       22

In [3]:
df = df[df['request']!='/robots.txt']
df = df[df['request']!='/favicon.ico']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4917 entries, 0 to 5865
Data columns (total 6 columns):
ip           4917 non-null object
date_time    4917 non-null object
request      4917 non-null object
step         4917 non-null int64
session      4917 non-null int64
user_id      4917 non-null int64
dtypes: int64(3), object(3)
memory usage: 268.9+ KB


In [4]:
sessions = df.groupby(['user_id'])['request'].apply(list)
print(sessions.head(10))

user_id
2                         [/code/Global/code/menu.html]
3      [/, /guarantee.html, /more.html, /services.html]
5                            [/code/Ultra/services.htm]
6     [/richlands, /richlands/, /richlands/fileuploa...
7     [/newfarm, /newfarm/specials/, /richlands, /ri...
9                        [/code/Global/code/oform.html]
10    [/cgi-bin/FormMail.pl, /cgi-bin/FormMail.pl, /...
11    [/newfarm/, /richlands/, /richlands/contact/, ...
12                                                  [/]
13                                    [/, /direct.html]
Name: request, dtype: object


In [5]:
from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
session_list = list(sessions)
results = list(apriori(session_list, min_support=0.03, min_confidence=0.1))
# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'/'}), support=0.4399271844660194, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/'}), confidence=0.4399271844660194, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/'}), support=0.15351941747572814, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/'}), confidence=0.15351941747572814, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/javascript/menu.js'}), support=0.16868932038834952, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/javascript/menu.js'}), confidence=0.16868932038834952, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/pricelist'}), support=0.10800970873786407, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/newfarm/pricelist'}), confidence=0.10800970873786407, lift=1.0)]), RelationRecord(items=frozenset({'/newfarm/pricelist/'}), support=0.1049757281553398, ordere

In [6]:
def convert_apriori_results_to_pandas_df(results):
    rules = []

    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
            rule_set.support, rule.confidence, rule.lift])

    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])
result_df = convert_apriori_results_to_pandas_df(results)
result_df.head(20)

Left_side                   Right_side   Support  \
0                                                          /  0.439927   
1                                                  /newfarm/  0.153519   
2                                /newfarm/javascript/menu.js  0.168689   
3                                         /newfarm/pricelist  0.108010   
4                                        /newfarm/pricelist/  0.104976   
5                                             /services.html  0.141383   
6                  /direct.html                            /  0.037621   
7                   /index.html                            /  0.030340   
8                             /                   /more.html  0.067961   
9                    /more.html                            /  0.067961   
10                     /newfarm                            /  0.035801   
11                    /newfarm/                            /  0.040655   
12  /newfarm/javascript/menu.js                            /  0.038835   
13                   /richlands                            /  0.031553   
14                  /richlands/                            /  0.030947   
15                            /               /services.html  0.118325   
16               /services.html                            /  0.118325   
17                   /more.html               /services.html  0.060073   
18               /services.html                   /more.html  0.060073   
19                     /newfarm                    /newfarm/  0.044903   

    Confidence      Lift  
0     0.439927  1.000000  
1     0.153519  1.000000  
2     0.168689  1.000000  
3     0.108010  1.000000  
4     0.104976  1.000000  
5     0.141383  1.000000  
6     0.704545  1.601505  
7     0.675676  1.535881  
8     0.154483  1.831565  
9     0.805755  1.831565  
10    0.702381  1.596585  
11    0.264822  0.601968  
12    0.230216  0.523304  
13    0.658228  1.496220  
14    0.653846  1.486260  
15    0.268966  1.902383  
16    0.836910  1.902383  
17    0.712230  5.037577  
18    0.424893  5.037577  
19    0.880952  5.738378

In [7]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
result_df.head(50)

Left_side  \
170           /richlands,/richlands/javascript/menu.js   
164              /newfarm/specials/,/newfarm/pricelist   
237  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
229  /newfarm/specials/,/newfarm/pricelist/,/newfar...   
256  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
225  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
86                                       /richlands/,/   
85                                        /richlands,/   
167             /newfarm/specials/,/newfarm/pricelist/   
171          /richlands/,/richlands/javascript/menu.js   
68                                         /richlands/   
67                                          /richlands   
169                             /richlands,/richlands/   
146     /newfarm/specials/,/newfarm/javascript/menu.js   
72                       /richlands/javascript/menu.js   
71                                         /richlands/   
69                                          /richlands   
70                       /richlands/javascript/menu.js   
174            /newfarm/,/newfarm/javascript/menu.js,/   
77                                         /newfarm/,/   
145      /newfarm/specials,/newfarm/javascript/menu.js   
163               /newfarm/pricelist,/newfarm/specials   
236  /newfarm/pricelist,/newfarm/pricelist/,/newfar...   
166              /newfarm/pricelist/,/newfarm/specials   
224  /newfarm/pricelist,/newfarm/specials,/newfarm/...   
255  /newfarm/pricelist,/newfarm/pricelist/,/newfar...   
228  /newfarm/pricelist/,/newfarm/specials,/newfarm...   
80                       /newfarm/javascript/menu.js,/   
65                                   /newfarm/specials   
66                                  /newfarm/specials/   
35                                    /newfarm/contact   
36                                   /newfarm/contact/   
246  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
213  /newfarm/specials/,/newfarm/pdf/Web_Price_List...   
184  /newfarm/pricelist,/newfarm/javascript/menu.js...   
257  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
180  /newfarm/javascript/menu.js,/newfarm/pdf/Web_P...   
109          /newfarm/pdf/Web_Price_List.pdf,/newfarm/   
221  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
240  /newfarm/pricelist,/newfarm/javascript/menu.js...   
112                       /newfarm/pricelist,/newfarm/   
251  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
217  /newfarm/pricelist,/newfarm/specials,/newfarm/...   
201    /newfarm/specials/,/newfarm/pricelist,/newfarm/   
196  /newfarm/pricelist,/newfarm/pdf/Web_Price_List...   
124  /newfarm/pdf/Web_Price_List.pdf,/newfarm/javas...   
204  /newfarm/pricelist,/newfarm/pdf/Web_Price_List...   
233  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
130     /newfarm/pricelist,/newfarm/javascript/menu.js   
148  /newfarm/pricelist,/newfarm/pdf/Web_Price_List...   

                        Right_side   Support  Confidence       Lift  
170                    /richlands/  0.033981    0.982456  20.757535  
164              /newfarm/specials  0.032767    0.830769  20.744056  
237              /newfarm/specials  0.031553    0.825397  20.609909  
229              /newfarm/specials  0.030947    0.822581  20.539589  
256              /newfarm/specials  0.030947    0.822581  20.539589  
225              /newfarm/specials  0.030947    0.822581  20.539589  
86                      /richlands  0.030340    0.980392  20.451725  
85                     /richlands/  0.030340    0.961538  20.315582  
167              /newfarm/specials  0.031553    0.812500  20.287879  
171                     /richlands  0.033981    0.933333  19.470042  
68                      /richlands  0.043083    0.910256  18.988640  
67                     /richlands/  0.043083    0.898734  18.988640  
169  /richlands/javascript/menu.js  0.033981    0.788732  18.838130  
146              /newfarm/specials  0.035194    0.753247  18.808343  
72                     /richlands/  

In [8]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 170 to 12
Data columns (total 5 columns):
Left_side     260 non-null object
Right_side    260 non-null object
Support       260 non-null float64
Confidence    260 non-null float64
Lift          260 non-null float64
dtypes: float64(3), object(2)
memory usage: 12.2+ KB


In [9]:
result_df

Left_side  \
170           /richlands,/richlands/javascript/menu.js   
164              /newfarm/specials/,/newfarm/pricelist   
237  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
229  /newfarm/specials/,/newfarm/pricelist/,/newfar...   
256  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
225  /newfarm/specials/,/newfarm/pricelist,/newfarm...   
86                                       /richlands/,/   
85                                        /richlands,/   
167             /newfarm/specials/,/newfarm/pricelist/   
171          /richlands/,/richlands/javascript/menu.js   
68                                         /richlands/   
67                                          /richlands   
169                             /richlands,/richlands/   
146     /newfarm/specials/,/newfarm/javascript/menu.js   
72                       /richlands/javascript/menu.js   
71                                         /richlands/   
69                                          /richlands   
70                       /richlands/javascript/menu.js   
174            /newfarm/,/newfarm/javascript/menu.js,/   
77                                         /newfarm/,/   
145      /newfarm/specials,/newfarm/javascript/menu.js   
163               /newfarm/pricelist,/newfarm/specials   
236  /newfarm/pricelist,/newfarm/pricelist/,/newfar...   
166              /newfarm/pricelist/,/newfarm/specials   
224  /newfarm/pricelist,/newfarm/specials,/newfarm/...   
255  /newfarm/pricelist,/newfarm/pricelist/,/newfar...   
228  /newfarm/pricelist/,/newfarm/specials,/newfarm...   
80                       /newfarm/javascript/menu.js,/   
65                                   /newfarm/specials   
66                                  /newfarm/specials/   
..                                                 ...   
88                                  /newfarm,/newfarm/   
47                         /newfarm/javascript/menu.js   
48                                  /newfarm/specials/   
33                                           /newfarm/   
34                                  /newfarm/specials/   
22                         /newfarm/javascript/menu.js   
21                                            /newfarm   
75                           /more.html,/services.html   
81                /newfarm,/newfarm/javascript/menu.js   
175     /newfarm/javascript/menu.js,/newfarm,/newfarm/   
16                                      /services.html   
15                                                   /   
9                                           /more.html   
8                                                    /   
78                                  /newfarm,/newfarm/   
6                                         /direct.html   
87                              /richlands,/richlands/   
10                                            /newfarm   
7                                          /index.html   
13                                          /richlands   
14                                         /richlands/   
1                                                        
5                                                        
4                                                        
3                                                        
2                                                        
0                                                        
84               /newfarm/javascript/menu.js,/newfarm/   
11                                           /newfarm/   
12                         /newfarm/javascript/menu.js   

                        Right_side   Support  Confidence       Lift  
170                    /richlands/  0.033981    0.982456  20.757535  
164              /newfarm/specials  0.032767    0.830769  20.744056  
237              /newfarm/specials  0.031553    0.825397  20.609909  
229              /newfarm/specials  0.030947    0.822581  20.539589  
256              /newfarm/specials  0.030947    0.822581  20.539589  
225              /newfarm/sp